In [1]:
%reset -f

In [2]:
import pandas as pd
from sssom.parsers import parse_sssom_table
from sssom import compare_dataframes
from sssom.parsers import split_dataframe
from sssom.util import MappingSetDataFrame
from os.path import join
from oaklib import OntologyResource
from oaklib.implementations import SqlImplementation
import textdistance

In [3]:
lexmatch_file = "../mappings/mondo-sources-all-lexical.sssom.tsv"
mondo_sssom = "../ontology/tmp/mondo.sssom.tsv"
dir_name = "dataframes"

In [4]:
# Functions

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_label.lower(), x.object_label.lower()\
                    if pd.notnull(x.object_label) else "99"\
                 ),\
                axis=1,\
            )\
           )

In [5]:
%%time
msdf_lex = parse_sssom_table(lexmatch_file)
msdf_mondo = parse_sssom_table(mondo_sssom)

# Use OAK to get 'object_label'
ontology_resource = OntologyResource(slug='../ontology/tmp/merged.db', local=True)
oi = SqlImplementation(ontology_resource)

CPU times: user 1min 43s, sys: 3.09 s, total: 1min 47s
Wall time: 1min 56s


In [6]:
# msdf_mondo.df[msdf_mondo.df['object_id'].str.contains('ICD')]
# "ICD10CM", "MONDO"
condition_1 = msdf_mondo.df['object_id'].str.contains("ICD10CM")
condition_2 = msdf_mondo.df['subject_id'].str.contains("MONDO")
condition_3 = msdf_mondo.df['object_id'].str.contains("MONDO")
condition_4 = msdf_mondo.df['subject_id'].str.contains("ICD10CM")
msdf_mondo.df = msdf_mondo.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))
msdf_mondo.df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
51,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Unspecified,Precocious puberty
111,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.3,Unspecified,Discordant ventriculoarterial connection
113,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.5,Unspecified,Discordant atrioventricular connection
202,MONDO:0000226,mineral metabolism disease,skos:exactMatch,ICD10CM:E83,Unspecified,Disorders of mineral metabolism
222,MONDO:0000237,erysipeloid,skos:exactMatch,ICD10CM:A26,Unspecified,Erysipeloid


In [7]:
# msdf_lex.df[msdf_lex.df['object_id'].str.contains('ICD')]
condition_1 = msdf_lex.df['object_id'].str.contains("ICD10CM")
condition_2 = msdf_lex.df['subject_id'].str.contains("MONDO")
condition_3 = msdf_lex.df['object_id'].str.contains("MONDO")
condition_4 = msdf_lex.df['subject_id'].str.contains("ICD10CM")
msdf_lex.df = msdf_lex.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
msdf_lex.df.head()


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
128740,ICD10CM:A00,Cholera,skos:exactMatch,MONDO:0015766,cholera,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,cholera
128746,ICD10CM:A01.0,Typhoid fever,skos:exactMatch,MONDO:0005619,typhoid fever,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,typhoid fever
128747,ICD10CM:A01.0,Typhoid fever,skos:exactMatch,MONDO:0005619,typhoid fever,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,typhoid fever
128751,ICD10CM:A01.1,Paratyphoid fever A,skos:closeMatch,MONDO:0018626,paratyphoid fever,semapv:LexicalMatching,oaklib,0.500000,rdfs:label,oio:hasRelatedSynonym,paratyphoid fever a
128755,ICD10CM:A01.2,Paratyphoid fever B,skos:closeMatch,MONDO:0018626,paratyphoid fever,semapv:LexicalMatching,oaklib,0.500000,rdfs:label,oio:hasRelatedSynonym,paratyphoid fever b


In [8]:
duplicate_df_lex = msdf_lex.df.copy()
duplicate_df_mondo = msdf_mondo.df.head().copy()

duplicate_df_lex = duplicate_df_lex.rename(columns={"subject_id": "object_id","object_id": "subject_id" })
duplicate_df_mondo = duplicate_df_mondo.rename(columns={"subject_id": "object_id","object_id": "subject_id" })


msdf_lex.df = pd.concat([duplicate_df_lex, msdf_lex.df]).drop_duplicates()
msdf_mondo.df = pd.concat([duplicate_df_mondo, msdf_mondo.df]).drop_duplicates()
msdf_mondo.df.head()

,object_id,subject_label,predicate_id,subject_id,mapping_justification,object_label
51,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Unspecified,Precocious puberty
111,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.3,Unspecified,Discordant ventriculoarterial connection
113,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.5,Unspecified,Discordant atrioventricular connection
202,MONDO:0000226,mineral metabolism disease,skos:exactMatch,ICD10CM:E83,Unspecified,Disorders of mineral metabolism
222,MONDO:0000237,erysipeloid,skos:exactMatch,ICD10CM:A26,Unspecified,Erysipeloid


In [9]:
comparison_ms_diff = compare_dataframes(msdf_mondo.df, msdf_lex.df)
comparison_df = comparison_ms_diff.combined_dataframe
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
comparison_df.head()

,object_id,subject_label,predicate_id,subject_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
58813,ICD10CM:B67.5,cystic echinococcosis,skos:broadMatch,MONDO:0018408,Unspecified,Echinococcus multilocularis infection of liver,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
12914,ICD10CM:B00,herpes simplex infectious disease,skos:exactMatch,MONDO:0004609,Unspecified,Herpesviral [herpes simplex] infections,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
55965,ICD10CM:B67.7,alveolar echinococcosis,skos:broadMatch,MONDO:0017282,Unspecified,"Echinococcus multilocularis infection, unspeci...",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
53470,ICD10CM:C53.1,papillary carcinoma of the cervix uteri,skos:broadMatch,MONDO:0016285,Unspecified,Malignant neoplasm of exocervix,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
16585,ICD10CM:B38.3,coccidioidomycosis,skos:broadMatch,MONDO:0005706,Unspecified,Cutaneous coccidioidomycosis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [10]:
mappings = ["LEXMATCH", "MONDO_MAPPINGS"]
unmapped_df = comparison_df[
    (comparison_df['comment'].str.contains("|".join(mappings)))
]

unmapped_df.head()

,object_id,subject_label,predicate_id,subject_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
58813,ICD10CM:B67.5,cystic echinococcosis,skos:broadMatch,MONDO:0018408,Unspecified,Echinococcus multilocularis infection of liver,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
12914,ICD10CM:B00,herpes simplex infectious disease,skos:exactMatch,MONDO:0004609,Unspecified,Herpesviral [herpes simplex] infections,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
55965,ICD10CM:B67.7,alveolar echinococcosis,skos:broadMatch,MONDO:0017282,Unspecified,"Echinococcus multilocularis infection, unspeci...",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
53470,ICD10CM:C53.1,papillary carcinoma of the cervix uteri,skos:broadMatch,MONDO:0016285,Unspecified,Malignant neoplasm of exocervix,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
16585,ICD10CM:B38.3,coccidioidomycosis,skos:broadMatch,MONDO:0005706,Unspecified,Cutaneous coccidioidomycosis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [11]:
# Let reviewers check if this makes sense or no.
unmapped_lex_exact = unmapped_df[(unmapped_df['comment'] == 'LEXMATCH') & (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_lex_exact.to_csv(join(dir_name, "unmapped_lex_exact.tsv"), sep='\t', index = False)
unmapped_lex_exact.head()


,object_id,subject_label,predicate_id,subject_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
131453,MONDO:0004782,Diabetes insipidus,skos:exactMatch,ICD10CM:E23.2,semapv:LexicalMatching,diabetes insipidus,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,diabetes insipidus
130647,ICD10CM:C92.0,Acute myeloblastic leukemia,skos:exactMatch,MONDO:0005224,semapv:LexicalMatching,acute myeloblastic leukemia without maturation,LEXMATCH,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,acute myeloblastic leukemia m1
130647,MONDO:0005224,Acute myeloblastic leukemia,skos:exactMatch,ICD10CM:C92.0,semapv:LexicalMatching,acute myeloblastic leukemia without maturation,LEXMATCH,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,acute myeloblastic leukemia m1
134660,MONDO:0005990,Acute tracheitis,skos:exactMatch,ICD10CM:J04.1,semapv:LexicalMatching,tracheitis,LEXMATCH,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,acute tracheitis
128968,MONDO:0005692,Cat-scratch disease,skos:exactMatch,ICD10CM:A28.1,semapv:LexicalMatching,cat-scratch disease,LEXMATCH,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,cat-scratch fever


In [12]:
# Inspect why these are missing from SSSOM mappings

unmapped_mondo_exact = unmapped_df[(unmapped_df['comment'] == 'MONDO_MAPPINGS')& (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_mondo_exact['object_label'] = unmapped_mondo_exact['object_id'].apply(lambda x: oi.label(x))
# Add distances
# Add Levenshtein distance [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "levenshtein_dist", textdistance.levenshtein.distance)
# Add Jaccard Index [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "jaccard_index", textdistance.jaccard.distance)
# Add Monge-Elkan Distance [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "monge_elkan", textdistance.monge_elkan.distance)
unmapped_mondo_exact.to_csv(join(dir_name, "unmapped_mondo_exact.tsv"), sep='\t', index = False)
unmapped_mondo_exact.head()


/var/folders/j5/bzs7jjf12j998fm88l0ssg740000gp/T/ipykernel_84239/653029316.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmapped_mondo_exact['object_label'] = unmapped_mondo_exact['object_id'].apply(lambda x: oi.label(x))


,object_id,subject_label,predicate_id,subject_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string,levenshtein_dist,jaccard_index,monge_elkan
12914,ICD10CM:B00,herpes simplex infectious disease,skos:exactMatch,MONDO:0004609,Unspecified,Herpesviral [herpes simplex] infections,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,23,0.285714,1.971534
70393,ICD10CM:T20-T25,burn,skos:exactMatch,MONDO:0043519,Unspecified,"Burns and corrosions of external body surface,...",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,70,0.945946,1.750000
14948,ICD10CM:M19,osteoarthritis,skos:exactMatch,MONDO:0005178,Unspecified,Other and unspecified osteoarthritis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,22,0.611111,1.928571
6731,ICD10CM:F10-F19,substance-related disorder,skos:exactMatch,MONDO:0002494,Unspecified,Mental and behavioral disorders due to psychoa...,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,63,0.653333,1.961538
13078,ICD10CM:B06,rubella,skos:exactMatch,MONDO:0004656,Unspecified,Rubella [German measles],MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,17,0.708333,1.857143


In [13]:
combined_msdf = MappingSetDataFrame(df=comparison_df, prefix_map=msdf_lex.prefix_map, metadata=msdf_lex.metadata)
combined_msdf.df.head()

,object_id,subject_label,predicate_id,subject_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
58813,ICD10CM:B67.5,cystic echinococcosis,skos:broadMatch,MONDO:0018408,Unspecified,Echinococcus multilocularis infection of liver,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
12914,ICD10CM:B00,herpes simplex infectious disease,skos:exactMatch,MONDO:0004609,Unspecified,Herpesviral [herpes simplex] infections,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
55965,ICD10CM:B67.7,alveolar echinococcosis,skos:broadMatch,MONDO:0017282,Unspecified,"Echinococcus multilocularis infection, unspeci...",MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
53470,ICD10CM:C53.1,papillary carcinoma of the cervix uteri,skos:broadMatch,MONDO:0016285,Unspecified,Malignant neoplasm of exocervix,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
16585,ICD10CM:B38.3,coccidioidomycosis,skos:broadMatch,MONDO:0005706,Unspecified,Cutaneous coccidioidomycosis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [14]:
%%time
df_dict = split_dataframe(combined_msdf)

CPU times: user 2.92 s, sys: 47.2 ms, total: 2.97 s
Wall time: 6.4 s


In [15]:
mondo_icd_list = [x for x in list(df_dict.keys()) if 'mondo' in x and "icd10" in x]
mondo_icd_list

['icd10cm_narrowmatch_mondo',
 'icd10cm_exactmatch_mondo',
 'icd10cm_closematch_mondo',
 'icd10cm_broadmatch_mondo',
 'mondo_narrowmatch_icd10cm',
 'mondo_exactmatch_icd10cm',
 'mondo_closematch_icd10cm',
 'mondo_broadmatch_icd10cm']

In [16]:
for match in mondo_icd_list:
    fn = match + ".tsv"
    df_dict[match].df.to_csv(join(dir_name,fn), sep='\t', index = False)

In [17]:
df_dict['mondo_exactmatch_icd10cm'].df

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string,comment
0,MONDO:0000022,Nocturnal enuresis,skos:exactMatch,ICD10CM:N39.44,nocturnal enuresis,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,nocturnal enuresis,LEXMATCH
1,MONDO:0000078,Congenital malformation syndromes predominantl...,skos:exactMatch,ICD10CM:Q87.0,acrocephalopolysyndactyly,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,rdfs:label,acrocephalopolysyndactyly,LEXMATCH
2,MONDO:0000088,Precocious puberty,skos:exactMatch,ICD10CM:E30.1,precocious puberty,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,precocious puberty,COMMON_TO_BOTH
3,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Precocious puberty,Unspecified,,NaN,,,,COMMON_TO_BOTH
4,MONDO:0000156,Craniosynostosis,skos:exactMatch,ICD10CM:Q75.0,trigonocephaly,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,trigonocephaly,LEXMATCH
...,...,...,...,...,...,...,...,...,...,...,...,...
4452,MONDO:0800029,Idiopathic pulmonary fibrosis,skos:exactMatch,ICD10CM:J84.112,interstitial lung disease 2,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,cryptogenic fibrosing alveolitis,COMMON_TO_BOTH
4453,MONDO:8000010,Antiphospholipid syndrome,skos:exactMatch,ICD10CM:D68.61,antiphospholipid syndrome,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,antiphospholipid syndrome,COMMON_TO_BOTH
4454,MONDO:8000010,Antiphospholipid syndrome,skos:exactMatch,ICD10CM:D68.61,antiphospholipid syndrome,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,antiphospholipid syndrome,COMMON_TO_BOTH
4455,MONDO:8000010,antiphospholipid syndrome,skos:exactMatch,ICD10CM:D68.61,Antiphospholipid syndrome,Unspecified,,NaN,,,,COMMON_TO_BOTH
